In [1]:
versioninfo()

Julia Version 0.5.0
Commit 3c9d753 (2016-09-19 18:14 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, haswell)


In [2]:
using OpenCL
using Gadfly

In [3]:
diff_kernel = "
__kernel void diff(__global const float *a,
__global float *diff,
int mesh_size)
    {
      int gid = get_global_id(0);
    diff[gid] = - 2. * a[gid] + a[(gid - 1) % mesh_size] + a[(gid + 1) % mesh_size];
    }
";

In [4]:
sum_kernel = "
   __kernel void sum(__global float *a,
__global const float *diff,
float step_size)
    {
      int gid = get_global_id(0);
    a[gid] += step_size * diff[gid];
    }
";

In [5]:
function initial_cond()
    a = zeros(Float32, (5000, 5000))
    a[Int(size(a)[1] / 2), Int(size(a)[2] / 2)] = 10
    return a
end

initial_cond (generic function with 1 method)

In [6]:
device, ctx, queue = cl.create_compute_context()

a_buff = cl.Buffer(Float32, ctx, (:rw, :copy), hostbuf=a)
diff_buff = cl.Buffer(Float32, ctx, :rw, length(a))

diff_p = cl.Program(ctx, source=diff_kernel) |> cl.build!
diff_k = cl.Kernel(diff_p, "diff")

sum_p = cl.Program(ctx, source=sum_kernel) |> cl.build!
sum_k = cl.Kernel(sum_p, "sum")

step_size = 0.01

for i in 1:10000
    queue(diff_k, size(a), nothing, a_buff, diff_buff, size(a)[1], length(a))
    queue(sum_k, size(a), nothing, a_buff, diff_buff, Float32(step_size))
end
r = cl.read(queue, a_buff);


LoadError: UndefVarError: a not defined

In [ ]:
r[2498:2502]

In [ ]:
plot(x=1:length(a), y=r, Geom.line)

In [ ]:
a